## Пробуем суммаризировать тексты

Используя модель bert, находим схожесть (cosine distance) между всеми предложениями текста.

Строим граф узлами которого являются предложения текста, а длина связей определена найденным ранее значением схожести, чем более схожы предложения, тем ближе они между собой в графе.

Далее алгоритмом Краскала находим минимальное оставное дерево в графе.

Ограничением при построении дерева служит указанное минимальное количество узлов.

Построение останавливается когда не останется деревьев с количеством узлов больше чем указано в ограничении.

Минимальное оставное дерево - это суммаризация текста.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as f
from transformers import AutoTokenizer, AutoModel

In [2]:
target = torch.tensor([[1, 2, 3, 4],
                        [1, 2, 3, 4]], dtype=float)
preds = torch.tensor([[1, 2, 3, 4],
                       [-1, -2, -3, -4]], dtype=float)

torch.cosine_similarity(preds, target, dim=1)

tensor([ 1.0000, -1.0000], dtype=torch.float64)

In [3]:
model_version = "cointegrated/rubert-tiny2"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_version)
model = AutoModel.from_pretrained(model_version)

In [6]:
model = model.eval()
model = model.to(device)

In [7]:
texts  = [
    'привет',
    'всем',
    'много',
    'лет'
]

In [8]:
encodings = tokenizer(
    texts, # the texts to be tokenized
    padding=True, # pad the texts to the maximum length (so that all outputs have the same length)
    return_tensors='pt' # return the tensors (not lists)
)

In [9]:
encodings = encodings.to(device)

In [10]:
# disable gradient calculations
with torch.no_grad():
    # get the model embeddings
    embeds = model(**encodings)

In [11]:
embeds = embeds[0]

In [12]:
embeds.shape

torch.Size([4, 3, 312])

In [13]:
torch.cosine_similarity(embeds[0], embeds[1])

tensor([0.8782, 0.8561, 0.9206])

In [14]:
MEANS = embeds.mean(dim=1)
MEANS.shape

torch.Size([4, 312])

In [15]:
torch.cosine_similarity(MEANS[0], MEANS[1], dim=0)

tensor(0.8926)

In [16]:
def get_similarity_text_matrix(texts, model, device):

    encodings = tokenizer(
        texts, # the texts to be tokenized
        padding=True, # pad the texts to the maximum length (so that all outputs have the same length)
        return_tensors='pt' # return the tensors (not lists)
    )

    encodings = encodings.to(device)

    # disable gradient calculations
    with torch.no_grad():
        # get the model embeddings
        embeds = model(**encodings)

    embeds = embeds[0]
    means = embeds.mean(dim=1)

    texts_len = len(texts)
    matrix = np.eye(texts_len, dtype=float)

    for i in range(0, texts_len):
        for j in range(0, texts_len):
            if i != j:
                sim = torch.cosine_similarity(means[i], means[j], dim=0).item()
                matrix[i, j] = sim

    return matrix

In [17]:
def get_matrix_df(words, matrix):
    df = pd.DataFrame(matrix)
    df.columns = words
    df.insert(0, '', words)

    return df

In [19]:
text = "всем привет df8 как 23 ваши дела"
words = text.split(' ')

matrix = get_similarity_text_matrix(words, model, device)
get_matrix_df(words, matrix)

,,всем,привет,df8,как,23,ваши,дела
0,всем,1.000000,0.912513,0.532773,0.897429,0.804247,0.918380,0.896599
1,привет,0.912513,1.000000,0.546540,0.907677,0.804162,0.923882,0.891256
2,df8,0.532773,0.546540,1.000000,0.564786,0.574964,0.562888,0.538052
3,как,0.897429,0.907677,0.564786,1.000000,0.833596,0.921991,0.916361
4,23,0.804247,0.804162,0.574964,0.833596,1.000000,0.818882,0.811977
5,ваши,0.918380,0.923882,0.562888,0.921991,0.818882,1.000000,0.925869
6,дела,0.896599,0.891256,0.538052,0.916361,0.811977,0.925869,1.000000
